<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Тригонометрическая интерполяция

### Modules - Curve Fitting
<section class="post-meta">
By Jonas Tjemsland, Andreas Krogen, Håkon Ånes and Jon Andreas Støvneng
</section>
Last edited: January 26th 2018 

___

### Вступление

Тригонометрический полином степени $K$ может быть записан как сумма синусов и косинусов заданных периодов

$$
P_K(t)=a_0 + \sum_{k=1}^Ka_k\cos(kt)+\sum_{k=1}^Kb_k\sin(kt).
$$

Эта форма интерполяции особенно подходит для периодических функций. Цель тригонометрической интерполяции состоит в вычислении коэффициентов $2n+1$ $a_0,a_1,...,a_K,b_1,b_2,...,b_K$, таких, что функция $P_K(t)$ проходит через заданный набор точек данных. Здесь мы предполагаем, что точки данных расположены на равном расстоянии друг от друга. Одним из способов решения этой проблемы является выполнение полиномиальной интерполяции на единичной окружности в комплексной плоскости, т.е. выполнение полиномиальной интерполяции на

$$
P(t)=\sum c_kz^t,
$$

где $c_k=a_k+ib_k$ и $z=e^{it}$. Этот подход используется в работе [1]. Таким образом, обсуждение полиномиальной интерполяции в некоторой степени может быть рассмотрено с использованием тригонометрической интерполяции. Однако тригонометрическая интерполирующая функция $P(t)$, очевидно, не уникальна, поскольку $e^{it} = e^{i(t + 2\pi)} = e^{i(t + 4\pi)} =$ ..., что требует дальнейших манипуляций.

Для обсуждения полиномиальной интерполяции (Лагранжа, Чебышева, Ньютона, явления Рунге и т.д.) Мы предлагаем заглянуть в нашу записную книжку [Polynomial Interpolation](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/polynomial_interpolation.ipynb). Здесь мы подходим к решению с использованием дискретных преобразований Фурье (DFT), как в [2]. Следовательно, некоторые базовые знания о DFT будут полезны, которые, например, можно просмотреть в другой записной книжке под названием [Discrete Fourier Transform and Fast Fourier Transform](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/discrete_fourier_transform.ipynb).

Тогда ладно. Как всегда, мы начинаем с импорта необходимых библиотек и устанавливаем общие параметры рисунка.

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy.fftpack as fft
%matplotlib inline

In [ ]:
# Приведение унитарных чисел к действительным числам приведет к ошибкам
# из-за ошибок числового округления. Поэтому мы отключаем 
# предупреждающие сообщения.
import warnings
warnings.filterwarnings('ignore')

# Set common figure parameters
newparams = {'figure.figsize': (16, 6), 'axes.grid': True,
             'lines.linewidth': 1.5, 'lines.markersize': 10,
             'font.size': 14}
plt.rcParams.update(newparams)

### Обобщение интерполяции

Предположим,что заданный набор $n$ точек данных $(t_i, x_i)$, где $i=0,1,..., n-1$, и пусть $f_0(t), f_1(t),...,f_{n-1}(t)$ заданы функции $t$. Теперь мы требуем, чтобы матрица $n\times n$ 

$$
A=\begin{bmatrix}
f_0(t_0)&\cdots&f_0(t_{n-1})\\
\vdots&\ddots&\vdots\\
f_{n-1}(t_0)&\cdots&f_{n-1}(t_{n-1})
\end{bmatrix},
$$

была унитарной. То есть, $A^{-1}=\overline{A}^T$. Если $A$ вещественная, то $A^{-1}=A^T$ и матрица ортогональна. Теперь мы определяем $\vec y=A\vec x$, что подразумевает, что $\vec x=\overline A^T \vec y$. Записываем в терминах сумм $x_j=\sum_{k=0}^{n-1}y_k\overline{f_k(t_j)}.$
Таким образом, функция

$$
F(t)=\sum_{k=0}^{n-1}y_k\overline{f_k(t)},
$$

интерполирует заданный набор данных! Если $A$ является вещественной $n\times n$ ортогональной матрицей, то

$$
F(t)=\sum_{k=0}^{n-1}y_kf_k(t).
$$

Поскольку DFT можно записать в виде унитарной матрицы $F_n$, мы можем использовать ее для интерполяции заданного набора данных.

### От дискретного преобразования Фурье к тригонометрической интерполяции

Из ранее упомянутой записной книжки по DFT у нас есть отличный алгоритм быстрого преобразования Фурье (FFT) для вычисления

$$
x_j = \frac{1}{n}\sum_{k=0}^{n-1}y_ke^{i2\pi kj/n},
$$

который легко можно переписать как

$$
x_j=\sum_{k=0}^{n-1}y_k\frac{\exp\left[\frac{i2\pi k(t_j-c)}{d-c}\right]}{n},
$$

где $t_j=c+j(d-c)/n$ для $j = 0, 1, ..., n-1$ для заданного интервала $[c,d]$. Предположим, что $\vec x = (x_0,x_1,...,x_{n-1})$ - это известный набор точек данных. Затем сложная функция

$$
Q(t)=\frac{1}{n}\sum_{k=0}^{n-1}y_k\exp\left[\frac{i2\pi k(t-c)}{d-c}\right]
$$

удовлетворяет $Q(t_j)=x_j$ для $j=0,1,...,n-1$. Другими словами, $Q(t)$ интерполирует множество $\{(x_j,t_j); \,j=0,1,...,n-1\}$, а коэффициенты интерполяции определяются преобразованием Фурье! Обратите внимание, что $t_i$ должен располагаться на равном расстоянии друг от друга. Как правило, это не требуется, но это упрощает дело.

In [ ]:
def trigInterp(x, c=0, d=1, N=100):
    """Вычисляет тригонометрический полином степени n-1, который интерполирует
    набор из n комплексных (или реальных) точек данных. Точки данных могут быть записаны
    как (t_i,x_i), i=0,1,2,..., где t_i равномерно распределены на интервале
    [c,d].
    
    :x: complex or float numpy array. Data points.
    :c: float. Start value t-axis. t[0].
    :d: float. End value t-axis. t[N-1].
    :N: int. Количество оценок функции интерполирующей функции.
    :returns: complex64 numpy array. Вторая ось интерполирующей функции.
    """
    t = np.linspace(c, d, N)  # t-values, first axis
    n = len(x)  # Number of data points
    y = fft.fft(x)  # Interpolating coefficients
    
    # Evaluate sum
    Q = np.zeros(N, np.complex64)
    for k in range(0, n):
        Q = Q + y[k]*np.exp(2j*np.pi*k*(t-c)/(d-c))
        
    return Q/n

Давайте попробуем на примере. Для простоты мы позволяем точкам данных $\vec x$ быть вещественными, хотя приведенная выше функция может использоваться и для еомплексных точек данных.

In [ ]:
x = np.random.randint(-10, 10, 10)  # Data points
c, d = 1, 2  # Interval for t
N = 200  # Number of function evaluations

# Вычислит кривую интерполяции
Q = trigInterp(x, c, d, N)

# Plot results
plt.figure()
plt.plot(np.linspace(c, d, len(Q)), Q, label=r'Interpolation curve, $Q(t)$')
plt.plot(np.linspace(c, d, len(x), False), x, 'mo', label='Data points, $x_k$')
plt.xlabel('$t$'), plt.ylabel('$x$'), plt.title('Trigonometric interpolation')
plt.legend();

### Улучшаем алгоритм – используем периодичность и удаление высоких частот

Используя формулу Эйлера, мы можем записать $Q(t)=P(t)+iI(t)$. В следующих обсуждениях мы будем предполагать, что элементы $x_j$ вещественны, так что $Q(t)=P(t)$, и пусть $y_k=a_k+ib_k$. Следующее обсуждение может быть применено и к $I(t)$. Из определения DFT ясно, что $y_0$ должен быть вещественным и $y_{n-k} = \overline{ y_k}$, если каждый элемент в $x$ вещественен. Кроме того, мы можем использовать периодичность $\cos(2\pi n-r)=\cos(r)$ и $\sin(2\pi n-r)=-\sin(r)$, чтобы сделать более эффективный алгоритм вычисления интерполяционной кривой. К настоящему времени должно быть ясно, что тригонометрический интерполирующий полином не уникален, как объясняется во введении.

Кривая интерполяции становится

$$
Q(t)=P_n(t)=\frac{1}{n}\sum_{k=0}^{n-1}\left(a_k \cos\frac{2\pi k(t-c)}{d-c}-b_k \sin\frac{2\pi k(t-c)}{d-c}\right).
$$

Если $n$ четно, мы можем упростить ее

$$
P_{n,\text{even}}(t)=\frac{a_0}{n}+\frac{2}{n}\sum_{k=1}^{n/2-1}\left(a_k \cos\frac{2\pi k(t-c)}{d-c}-b_k \sin\frac{2\pi k(t-c)}{d-c}\right)+\frac{a_{n/2}}{\sqrt n}\cos \frac{n\pi(t-c)}{d-c},
$$

таким образом, вдвое сокращается количество необходимых вычислений. Обратите внимание, что этот интерполирующий полином не такой, как в предыдущем разделе. Все более короткие периоды не включены, и, таким образом, результат будет несколько более плавным. Если $n$ нечетно, кривая интерполяции становится

$$
P_{n,\text{odd}}(t)=\frac{a_0}{n}+\frac{2}{n}\sum_{k=1}^{(n-1)/2}\left(a_k \cos\frac{2\pi k(t-c)}{d-c}-b_k \sin\frac{2\pi k(t-c)}{d-c}\right).
$$

Эти выражения могут быть вычислены и нанесены на график, как и раньше. Однако мы можем сделать еще лучше. Мы будем рассматривать коэффициенты $P(t)$ как коэффициенты тригонометрического полинома порядка $N\geq n$. Другими словами, $P_n(t)=P_N(t)$, где мы устанавливаем $a_k=b_k=0$ для $k=n/2+1,n/2+1,...,N/2$ и $a_k+ib_k=y_k$ для $k=0,1,...,n/2$. Если мы выполним обратное преобразование Фурье $\frac{N}{n}P_N(t)$, мы получим $N$ точек данных, лежащих на $P(t)$! Давайте сделаем это и посмотрим, как оно работает.

In [ ]:
def fastTrigInterp(x,c,d,N=100):
    """ Вычисляет тригонометрический полином степени n/2 (n четно) или
    (n-1)/2 (n нечетно) , который интерполирует набор из n реальных точек данных. Точки
    данных могут быть записаны как (t_i,x_i), i=0,1,2,..., где t_i
    равномерно расположены на интервале [c,d].
    
    :x: complex or float numpy array. Data points.
    :c: float. Start value t-axis. t[0].
    :d: float. End value t-axis. t[N-1].
    :N: int. Number of function evaluations of the interpolating function.
    :returns: float numpy array. Second axis of the interpolating function.
    """
    n = len(x)  # Number of data points
    y = fft.fft(x)  # Interpolating coefficients
    
    # Interpolating coefficients as viewed as a
    # trigonometric polynomial of order N.
    yp = np.zeros(N, np.complex64)
    yp[0:int(n/2)] = y[0:int(n/2)]
    yp[N - int(n/2):N] = y[int(n/2):n]

    return np.real(fft.ifft(yp))*N/n

Давайте опробуем нашу новую функцию интерполяции на том же наборе данных, что и выше.

In [ ]:
# Calculate interpolation curve
xp = fastTrigInterp(x, c, d, N)

# Plot results
plt.figure()
plt.plot(np.linspace(c, d, N, False), xp, label=r'Interpolation curve, $Q(t)$')
plt.plot(np.linspace(c, d, len(x), False), x, 'mo', label='Data points, $x_k$')
plt.xlabel('$t$'), plt.ylabel('$x$'), plt.title('Trigonometric interpolation')
plt.legend();

Используя периодичность точек данных и исключая более высокие частоты, мы ясно видим, что кривая интерполяции $Q(t)$ является более гладкой и, следовательно, соответствует точкам данных "лучше", чем наша предыдущая кривая $Q(t)$.

### Подгонка по методу наименьших квадратов

Когда количество точек данных становится большим, бесполезно подгонять модель в поисках избыточной сложности. Мы часто вместо этого заинтересованы в подгонке аппроксимированной кривой. Это часто делается при анализе измерений.

Задача наименьших квадратов [4] для этой установки может быть сформулирована следующим образом. Учитывая линейную систему $A_m\vec x=\vec y$ уравнений $m$, нужно найти вектор $\vec x$, который минимизирует $||\vec y-A\vec x||$. Другими словами, у нас есть противоречивая система уравнений $m$, которую мы хотим решить "как можно лучше". Можно показать, что для каждой линейной системы $A_m\vec x=\vec y$ соответствующая нормальная система $A_m^TA_m\vec x = A_m^T \vec y$ согласована. Более того, все решения нормальной системы являются решениями наименьших квадратов $A_m\vec x=\vec y$.

Давайте применим это к приведенной выше обобщенной интерполяции. $A_m$ - это матрица первых $m$ строк $A$. Поскольку $A$ унитарна (и ортогональна, если $A$ вещественна), векторы столбцов и векторы строк $A$ образуют ортонормированные множества. Аналогично, векторы столбцов и векторы строк $A_m$ будут образовывать ортонормированные множества. Таким образом, нормальное уравнение становится

$$
A_m^TA_m\vec x = I\vec x = \vec x = A_m^T \vec y.
$$

Это означает, что решение наименьших квадратов для $F_n(t)=\sum_{k=0}^{n-1}y_kf_k(t)$, использующее только первые $m$ уравнений, является

$$
F_m(t)=\sum_{k=0}^{m-1}y_kf_k(t).
$$

Для случая тригонометрической интерполяции это означает, что решение наименьших квадратов степени $m$ просто выполняется путем фильтрации членов $n-m$ самых высоких частот. Эти аргументы справедливы как для функций `trigInterp`, так и для функций `fastTrigInterp` выше. Теперь мы собираемся создать функцию, вычисляющую решение наименьших квадратов, основанное на последнем.

In [ ]:
def leastSquaresTrig(x, m, c=0, d=1, N=100):
    """Вычисляет соответствие степени тригонометрического полинома наименьших квадратов
    около m/2 из n реальных точек данных. Точки данных могут быть записаны в виде
    (t_i,x_i), i=0,1,2,..., где t_i равномерно распределены на интервале
    [c,d].
    
    :x: complex or float numpy array. Data points.
    :c: float. Start value t-axis. t[0].
    :d: float. End value t-axis. t[N-1].
    :N: int. Number of function evaluations of the fitted function.
    :returns: float numpy array. Second axis of the fitted function.
    """
    n = len(x)  # Number of data points
    if not 0<=m<=n<=N:
        raise ValueError('Is 0<=m<=n<=N??')
    y = fft.fft(x)  # Interpolating coefficients
    
    # Interpolating coefficients viewed as a
    # trigonometric polynomial of order N.
    yp = np.zeros(N, np.complex64)
    
    # Use only m lowest frequencies
    yp[0:int(m/2)] = y[0:int(m/2)]
    yp[N - int(m/2) + 1:N] = y[n - int(m/2) + 1:n]
    
    # If odd m, terms corresponding to m/2 has both a sine
    # and a cosine term
    if (m % 2):
        yp[int(m/2)] = y[int(m/2)]
    # If even m, terms corresponding to m/2 has only a cosine term
    else:
        yp[int(m/2)] = np.real(y[int(m/2)])
    if m<n and m>0:
        yp[N - int(m/2)] = yp[int(m/2)]
    
    return np.real(fft.ifft(yp))*N/n

Теперь мы выполняем тригонометрическую интерполяцию на тех же случайных точках данных, что и ранее, исключая увеличение числа частот. Как мы видели, это означает, что мы находим решения наименьших квадратов для тригонометрического полинома убывающей степени.

In [ ]:
plt.figure()
for m in [0, 2, 5, 8, 10]:
    # Calculate interpolation curve
    xp = leastSquaresTrig(x, m, c, d, N)
    # Plot results
    plt.plot(np.linspace(c, d, len(xp), False), xp, label=r'$m=%d$' % m)

plt.plot(np.linspace(c, d, len(x), False), x, 'mo')
plt.xlabel('$t$'), plt.ylabel('$x$')
plt.title('Least squares trigonometric fit to %d data points' % len(x))
plt.legend();

Итак, используя принципы, представленные в этой тетради, довольно легко создать алгоритм наименьших квадратов для подгонки тригонометрической функции с произвольными частотами.

### Ссылки
[1] Wikipedia: Trigonometric interpolation, https://en.wikipedia.org/wiki/Trigonometric_interpolation, 02.28.2016, Acquired May 4th 2016.  
[2] T. Sauer: Numerical Analysis, 2nd edition, Pearson 2013.  
[3] H. Anton, C. Rorres: Elementary Linear Algebra with Supplemental Applications, 11th edition, Wiley 2015.  
[4] Wikipedia: Least squares, https://en.wikipedia.org/wiki/Least_squares, Acquired May 6th 2016.